In [1]:
import pandas as pd
from pyqtree import Index
train = pd.read_csv('./train.csv')

In [546]:
# sample subset from the data set
data = train.sample(frac= 0.01)
data.shape

(291180, 6)

One of the difficulty of this data set is that it have lot of labels, let's see how many there are in our sample.

In [547]:
print 'Number of labels: %d' %len(data.place_id.unique())

Number of labels: 83345


In [548]:

# Add hours and day columns into data
# From the notebook EDA we see that the unit of time is 'second'

data['hour'] = (data.time/60) %24
data['day'] = (data.time/(60*24)) %7
data['month'] = ((data.time)/60*24*30) %12

data.hour = data.hour.apply(lambda x: round(x))
data.day = data.day.apply(lambda x: round(x))
data.month = data.month.apply(lambda x: round(x))
data.head()


,row_id,x,y,accuracy,time,place_id,hour,day,month
18909248,18909248,6.7762,3.0102,68,45007,4460929306,6.0,3.0,0.0
6954761,6954761,4.0223,8.7203,11,303844,5581742228,0.0,1.0,0.0
256006,256006,0.1626,3.3357,201,281717,2672645835,15.0,7.0,0.0
4824318,4824318,2.8897,0.2110,41,51231,5899723304,14.0,1.0,0.0
9734973,9734973,4.8039,4.0706,6,185085,1446642439,13.0,3.0,0.0


If we use the feature of 'hour', 'day' and 'month' directly, the accuracy will be even worse than 3%. The reason is that numbers have carinalit orders, while these features arer categorical data. To overcome this issue, we use one-hot-encoder.

In [549]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
time = pd.DataFrame(enc.fit_transform(data[['hour','day','month']]).toarray(), index= data.index)
data = pd.concat([data,time],axis = 1)
data.head(3)

,row_id,x,y,accuracy,time,place_id,hour,day,month,0,...,25,26,27,28,29,30,31,32,33,34
18909248,18909248,6.7762,3.0102,68,45007,4460929306,6.0,3.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6954761,6954761,4.0223,8.7203,11,303844,5581742228,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
256006,256006,0.1626,3.3357,201,281717,2672645835,15.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [550]:
# Normalize  accuracy

acc_mean = np.mean(data.accuracy)
acc_std = np.std(data.accuracy)


data.accuracy = data.accuracy.apply(lambda x: (x-acc_mean)/acc_std)

In [551]:
# filter out place id with very few check ins
count = data.groupby('place_id').count()
filtered_place_id = count[count.row_id >3].index.tolist()
filtered = data[(data.place_id.isin(filtered_place_id)) ]
filtered.shape

(194601, 44)

In [552]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(filtered[['x','y']+range(35)], filtered['place_id'], test_size=0.33, random_state=1024)
X_train.shape    

(130382, 37)

As a benchmark, we run knn on all area

In [553]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import time

# record [n_neighbors, accuracy, time spent]
new_result = []

nhds = [3]
for k in nhds:
    start = time.time()
    knn = KNeighborsClassifier(n_neighbors= k)
    model = knn.fit(X_train, y_train)
    pred = model.predict(X_test)
    done = time.time()
    new_result.append([k, np.sum([pred == y_test])/float(len(pred)),done - start ])

In [554]:
new_result

[[3, 0.020912813964714491, 88.61011600494385]]

Now we change our strategy. Given a validation point, we first find out training points close to it on the map. Then we train KNN on these neighbor training points. To be efficient, we use 
Quadtree data structure to save the data points.

In [555]:
#(xmin, ymin, xmax, ymax) quadtree instance
q_train =  Index(bbox=(-1, -1, 11, 11))


# construct  quadTree for X_train and X_test

import time
start = time.time()

# epison of neighbor
e = 0.1

for row in X_train.itertuples():
    q_train.insert(row.Index, (row.x-e, row.y-e, row.x+e, row.y+e))

#for row in X_test.itertuples():
#    q_test.insert(row.Index, (row.x-e, row.y-e, row.x+e, row.y+e))
done = time.time()
print 'Total time spent %d' %(done-start)

Total time spent 74


In [556]:
# A data frame to save prediction result
import numpy as np
result = pd.DataFrame(columns=['label','prediction'])
result['label'] = y_test
result['prediction'] = np.empty((len(result),0)).tolist()
result.head()

,label,prediction
10190674,7986235670,[]
11809242,3702031891,[]
17989285,3020259097,[]
16562206,2866492303,[]
28278988,9031150064,[]


In [557]:
# Run local KNN
from sklearn.neighbors import KNeighborsClassifier

start = time.time()
e = 0.3

for row in X_test.index:
    x = X_test.loc[row].x
    y = X_test.loc[row].y
    box = (x-e,y-e,x+e,y+e)
    
    # training
    train_row_id = q_train.intersect(box)
    knn = KNeighborsClassifier(n_neighbors= 5)
    model = knn.fit(X_train.loc[train_row_id], y_train.loc[train_row_id])
    
    #prediction
    prediction = model.predict(X_test.loc[row].values.reshape(1,-1))
    result.loc[row]['prediction'].append(prediction)
done = time.time()
print 'Total time spent %d' %(done-start)    

Total time spent 995


In [558]:
result['flat_pre'] = result.prediction.apply(lambda x: [item for sublist in x for item in sublist])

def f(row):
    return row['label'] in row['flat_pre']

result['accuracy'] = result.apply(f, axis = 1)

print 'The accuracy of QuadTree KNN: %f' %(np.sum(result.accuracy)/float(result.shape[0]))

The accuracy of QuadTree KNN: 0.096062


                 58000   116000   291180
Number_labels     38071       57000    83345
Plain KNN :       0.09     0.02   0.02 
QuadTree  :       0.23     0.11   0.09 
